In [1]:
#!pip install textstat

In [2]:
#!pip install langdetect

In [3]:
#!pip install spellchecker

In [4]:
#!pip install pyspellchecker

In [5]:
#!git clone https://github.com/brucewlee/lingfeat.git
#!python -m pip install -r lingfeatBASE/requirements.txt

In [6]:
#!python -m spacy download en_core_web_sm

In [7]:
#!pip install torch

The featuture extraction processes bases on https://github.com/brucewlee/lingfeat

In [8]:
import csv
import pickle
import string
import textstat

import pandas as pd
import numpy as np

from langdetect import detect
from spellchecker import SpellChecker
from tqdm import tqdm

In [9]:
#from lingfeatBASE.lingfeat import extractor
from lingfeat import extractor

In [10]:
text = "..."
lf = extractor.pass_text(text)
print(lf.preprocess())

{'n_token': 1, 'n_sent': 2}


### Import data

In [11]:
data = pd.read_csv('final_results_data.csv')
data

,Unnamed: 0,rid,qid,title,snippet,url,pos,clicked,event,event_data,event_type,full
0,0,1,d119db54-90aa-48b3-9035-eeda55edc38d,Incoming Webhooks — Mattermost 5.11 documentation,Use incoming webhooks to post messages to Matt...,https://docs.mattermost.com/developer/webhooks...,0,1,clk-result,"{\position\"":0}""",click result,Incoming Webhooks — Mattermost 5.11 documentat...
1,1,2,d119db54-90aa-48b3-9035-eeda55edc38d,“Hello” Is Facebook's New Android-Only Social ...,"Apr 22, 2015 ... Today, Hello is rolling out f...",https://techcrunch.com/2015/04/22/facebook-hello/,1,0,NaN,NaN,NaN,“Hello” Is Facebook's New Android-Only Social ...
2,2,3,d119db54-90aa-48b3-9035-eeda55edc38d,Walkthrough | HTTP Event Collector,HEC tokens are sent in the headers of incoming...,http://dev.splunk.com/view/event-collector/SP-...,4,0,NaN,NaN,NaN,Walkthrough | HTTP Event CollectorHEC tokens a...
3,3,4,d119db54-90aa-48b3-9035-eeda55edc38d,College Scholarships - Scholarships.com,Most college scholarships target incoming fres...,https://www.scholarships.com/financial-aid/col...,3,0,NaN,NaN,NaN,College Scholarships - Scholarships.comMost co...
4,4,5,d119db54-90aa-48b3-9035-eeda55edc38d,How to start sending email — Mailgun API docum...,"It's up to you, whatever you find easier is fi...",http://documentation.mailgun.com/en/latest/qui...,2,0,NaN,NaN,NaN,How to start sending email — Mailgun API docum...
...,...,...,...,...,...,...,...,...,...,...,...,...
10287,10287,26377,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,"Science, Technology, Engineering, and Math, in...",Department Offices that Support STEM Open ED F...,https://www.ed.gov/stem,1,1,clk-result,"{\position\"":\""1\""}""",click result,"Science, Technology, Engineering, and Math, in..."
10288,10288,26378,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Stem Kitchen SF,STEM Kitchen & Garden is a farm-to-table resta...,https://www.stemkitchensf.com/,6,0,NaN,NaN,NaN,Stem Kitchen SFSTEM Kitchen & Garden is a farm...
10289,10289,26379,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Stem Definition & Meaning - Merriam-Webster,Kids Definition of stem 1 : to develop as a c...,https://www.merriam-webster.com/dictionary/stem,7,0,NaN,NaN,NaN,Stem Definition & Meaning - Merriam-WebsterKid...
10290,10290,26380,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Driven By Stem,Stem Holdings has positioned itself as a leade...,https://www.drivenbystem.com/,8,0,NaN,NaN,NaN,Driven By StemStem Holdings has positioned its...


In [12]:
data.shape

(10292, 12)

In [13]:
## select the needed columns
data2 = data[['title', 'snippet', 'full', 'pos', 'clicked']]
data2

,title,snippet,full,pos,clicked
0,Incoming Webhooks — Mattermost 5.11 documentation,Use incoming webhooks to post messages to Matt...,Incoming Webhooks — Mattermost 5.11 documentat...,0,1
1,“Hello” Is Facebook's New Android-Only Social ...,"Apr 22, 2015 ... Today, Hello is rolling out f...",“Hello” Is Facebook's New Android-Only Social ...,1,0
2,Walkthrough | HTTP Event Collector,HEC tokens are sent in the headers of incoming...,Walkthrough | HTTP Event CollectorHEC tokens a...,4,0
3,College Scholarships - Scholarships.com,Most college scholarships target incoming fres...,College Scholarships - Scholarships.comMost co...,3,0
4,How to start sending email — Mailgun API docum...,"It's up to you, whatever you find easier is fi...",How to start sending email — Mailgun API docum...,2,0
...,...,...,...,...,...
10287,"Science, Technology, Engineering, and Math, in...",Department Offices that Support STEM Open ED F...,"Science, Technology, Engineering, and Math, in...",1,1
10288,Stem Kitchen SF,STEM Kitchen & Garden is a farm-to-table resta...,Stem Kitchen SFSTEM Kitchen & Garden is a farm...,6,0
10289,Stem Definition & Meaning - Merriam-Webster,Kids Definition of stem 1 : to develop as a c...,Stem Definition & Meaning - Merriam-WebsterKid...,7,0
10290,Driven By Stem,Stem Holdings has positioned itself as a leade...,Driven By StemStem Holdings has positioned its...,8,0


In [14]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10292 entries, 0 to 10291
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    10292 non-null  object
 1   snippet  10291 non-null  object
 2   full     10292 non-null  object
 3   pos      10292 non-null  int64 
 4   clicked  10292 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 402.2+ KB


In [15]:
data2['clicked'].value_counts()

clicked
0    5865
1    4427
Name: count, dtype: int64

In [16]:
data2['clicked'].value_counts(normalize = True)

clicked
0    0.56986
1    0.43014
Name: proportion, dtype: float64

In [17]:
title = data2.copy()
snippet = data2.copy()

In [20]:
title

,title,snippet,full,pos,clicked,extract
0,Incoming Webhooks — Mattermost 5.11 documentation,Use incoming webhooks to post messages to Matt...,Incoming Webhooks — Mattermost 5.11 documentat...,0,1,<lingfeat.extractor.pass_text object at 0x0000...
1,“Hello” Is Facebook's New Android-Only Social ...,"Apr 22, 2015 ... Today, Hello is rolling out f...",“Hello” Is Facebook's New Android-Only Social ...,1,0,<lingfeat.extractor.pass_text object at 0x0000...
2,Walkthrough | HTTP Event Collector,HEC tokens are sent in the headers of incoming...,Walkthrough | HTTP Event CollectorHEC tokens a...,4,0,<lingfeat.extractor.pass_text object at 0x0000...
3,College Scholarships - Scholarships.com,Most college scholarships target incoming fres...,College Scholarships - Scholarships.comMost co...,3,0,<lingfeat.extractor.pass_text object at 0x0000...
4,How to start sending email — Mailgun API docum...,"It's up to you, whatever you find easier is fi...",How to start sending email — Mailgun API docum...,2,0,<lingfeat.extractor.pass_text object at 0x0000...
...,...,...,...,...,...,...
10287,"Science, Technology, Engineering, and Math, in...",Department Offices that Support STEM Open ED F...,"Science, Technology, Engineering, and Math, in...",1,1,
10288,Stem Kitchen SF,STEM Kitchen & Garden is a farm-to-table resta...,Stem Kitchen SFSTEM Kitchen & Garden is a farm...,6,0,
10289,Stem Definition & Meaning - Merriam-Webster,Kids Definition of stem 1 : to develop as a c...,Stem Definition & Meaning - Merriam-WebsterKid...,7,0,
10290,Driven By Stem,Stem Holdings has positioned itself as a leade...,Driven By StemStem Holdings has positioned its...,8,0,


In [21]:
title['extract'] = ''
for i in range(len(title)):
    title['extract'][i] = extractor.pass_text(str(title['title'][i]))

In [22]:
snippet['extract'] = ''
for i in range(len(snippet)):
    snippet['extract'][i] = extractor.pass_text(str(snippet['snippet'][i]))

In [ ]:
data2['extract'] = ''
for i in range(len(data2)):
    data2['extract'][i] = extractor.pass_text(str(data2['full'][i]))

In [ ]:
print(data2['extract'][0])
print(data2['extract'][1])

In [23]:
title['preprocess'] = ''
for i in range(len(title)):
    title['preprocess'][i] = title['extract'][i].preprocess()

In [24]:
snippet['preprocess'] = ''
for i in range(len(snippet)):
    snippet['preprocess'][i] = snippet['extract'][i].preprocess()

In [ ]:
data2['preprocess'] = ''
for i in range(len(data2)):
    data2['preprocess'][i] = data2['extract'][i].preprocess()

In [ ]:
data2['WRich05_S'] = ''
for i in range(len(data2)):
    data2['WRich05_S'][i] = data2['extract'][i].WoKF_()
data2

,rid,qid,full,pos,clicked,extract,preprocess,WRich05_S
0,1,d119db54-90aa-48b3-9035-eeda55edc38d,Incoming Webhooks — Mattermost 5.11 documentat...,0,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 6}","{'WRich05_S': 2.158255100250244, 'WRich10_S': ..."
1,2,d119db54-90aa-48b3-9035-eeda55edc38d,“Hello” Is Facebook's New Android-Only Social ...,1,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 4}","{'WRich05_S': 2.363802380859852, 'WRich10_S': ..."
2,3,d119db54-90aa-48b3-9035-eeda55edc38d,Walkthrough | HTTP Event CollectorHEC tokens a...,4,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 41, 'n_sent': 5}","{'WRich05_S': 1.7482594717293978, 'WRich10_S':..."
3,4,d119db54-90aa-48b3-9035-eeda55edc38d,College Scholarships - Scholarships.comMost co...,3,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 3}","{'WRich05_S': 1.779820341616869, 'WRich10_S': ..."
4,5,d119db54-90aa-48b3-9035-eeda55edc38d,How to start sending email — Mailgun API docum...,2,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 5}","{'WRich05_S': 1.9575455449521542, 'WRich10_S':..."
...,...,...,...,...,...,...,...,...
10287,26377,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,"Science, Technology, Engineering, and Math, in...",1,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 27, 'n_sent': 5}","{'WRich05_S': 2.206125855445862, 'WRich10_S': ..."
10288,26378,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Stem Kitchen SFSTEM Kitchen & Garden is a farm...,6,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 22, 'n_sent': 2}","{'WRich05_S': 2.277161478996277, 'WRich10_S': ..."
10289,26379,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Stem Definition & Meaning - Merriam-WebsterKid...,7,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 22, 'n_sent': 3}","{'WRich05_S': 2.4866977632045746, 'WRich10_S':..."
10290,26380,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Driven By StemStem Holdings has positioned its...,8,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 19, 'n_sent': 2}","{'WRich05_S': 1.3848039507865906, 'WRich10_S':..."


In [31]:
ttrDef = {'SimpTTR_S':0, 'CorrTTR_S':0, 'BiLoTTR_S':0, 'UberTTR_S':0,'MTLDTTR_S':0}

In [ ]:
title['phrasal'] = ''
title['PartsOfSpeech'] = ''
title['TradFeats'] = ''
title['TTRatio'] = ''
title['Variation'] = ''
title['PDifficulty'] = ''
title['TRSF'] = ''
title['shallow'] = ''
for i in range(len(title)):
    title['phrasal'][i] = title['extract'][i].PhrF_()
    title['PartsOfSpeech'][i] = title['extract'][i].POSF_()
    title['TradFeats'][i] = title['extract'][i].TraF_()
    try:
        title['TTRatio'][i] = title['extract'][i].TTRF_()
    except:
        title['TTRatio'][i] = ttrDef
    title['Variation'][i] = title['extract'][i].VarF_()
    title['PDifficulty'][i] = title['extract'][i].PsyF_()
    title['TRSF'][i] = title['extract'][i].TrSF_()
    title['shallow'][i] = title['extract'][i].ShaF_()

In [ ]:
snippet['phrasal'] = ''
snippet['PartsOfSpeech'] = ''
snippet['TradFeats'] = ''
snippet['TTRatio'] = ''
snippet['Variation'] = ''
snippet['PDifficulty'] = ''
snippet['TRSF'] = ''
snippet['shallow'] = ''
for i in range(len(snippet)):
    snippet['phrasal'][i] = snippet['extract'][i].PhrF_()
    snippet['PartsOfSpeech'][i] = snippet['extract'][i].POSF_()
    snippet['TradFeats'][i] = snippet['extract'][i].TraF_()
    snippet['TTRatio'][i] = snippet['extract'][i].TTRF_()
    snippet['Variation'][i] = snippet['extract'][i].VarF_()
    snippet['PDifficulty'][i] = snippet['extract'][i].PsyF_()
    snippet['TRSF'][i] = snippet['extract'][i].TrSF_()
    snippet['shallow'][i] = snippet['extract'][i].ShaF_()

In [ ]:
title.to_csv('title_features.csv')
snippet.to_csv('snippet_features.csv')

In [ ]:
data2['phrasal'] = ''
for i in range(len(data2)):
    data2['phrasal'][i] = data2['extract'][i].PhrF_()
data2

,rid,qid,full,pos,clicked,extract,preprocess,WRich05_S,phrasal
0,1,d119db54-90aa-48b3-9035-eeda55edc38d,Incoming Webhooks — Mattermost 5.11 documentat...,0,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 6}","{'WRich05_S': 2.158255100250244, 'WRich10_S': ...","{'to_NoPhr_C': 10, 'as_NoPhr_C': 1.66666666666..."
1,2,d119db54-90aa-48b3-9035-eeda55edc38d,“Hello” Is Facebook's New Android-Only Social ...,1,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 4}","{'WRich05_S': 2.363802380859852, 'WRich10_S': ...","{'to_NoPhr_C': 14, 'as_NoPhr_C': 3.5, 'at_NoPh..."
2,3,d119db54-90aa-48b3-9035-eeda55edc38d,Walkthrough | HTTP Event CollectorHEC tokens a...,4,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 41, 'n_sent': 5}","{'WRich05_S': 1.7482594717293978, 'WRich10_S':...","{'to_NoPhr_C': 15, 'as_NoPhr_C': 3.0, 'at_NoPh..."
3,4,d119db54-90aa-48b3-9035-eeda55edc38d,College Scholarships - Scholarships.comMost co...,3,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 3}","{'WRich05_S': 1.779820341616869, 'WRich10_S': ...","{'to_NoPhr_C': 8, 'as_NoPhr_C': 2.666666666666..."
4,5,d119db54-90aa-48b3-9035-eeda55edc38d,How to start sending email — Mailgun API docum...,2,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 5}","{'WRich05_S': 1.9575455449521542, 'WRich10_S':...","{'to_NoPhr_C': 9, 'as_NoPhr_C': 1.8, 'at_NoPhr..."
...,...,...,...,...,...,...,...,...,...
10287,26377,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,"Science, Technology, Engineering, and Math, in...",1,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 27, 'n_sent': 5}","{'WRich05_S': 2.206125855445862, 'WRich10_S': ...","{'to_NoPhr_C': 17, 'as_NoPhr_C': 3.4, 'at_NoPh..."
10288,26378,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Stem Kitchen SFSTEM Kitchen & Garden is a farm...,6,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 22, 'n_sent': 2}","{'WRich05_S': 2.277161478996277, 'WRich10_S': ...","{'to_NoPhr_C': 9, 'as_NoPhr_C': 4.5, 'at_NoPhr..."
10289,26379,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Stem Definition & Meaning - Merriam-WebsterKid...,7,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 22, 'n_sent': 3}","{'WRich05_S': 2.4866977632045746, 'WRich10_S':...","{'to_NoPhr_C': 12, 'as_NoPhr_C': 4.0, 'at_NoPh..."
10290,26380,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Driven By StemStem Holdings has positioned its...,8,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 19, 'n_sent': 2}","{'WRich05_S': 1.3848039507865906, 'WRich10_S':...","{'to_NoPhr_C': 10, 'as_NoPhr_C': 5.0, 'at_NoPh..."


In [ ]:
data2['PartsOfSpeech'] = ''
for i in range(len(data2)):
    data2['PartsOfSpeech'][i] = data2['extract'][i].POSF_()
data2

,rid,qid,full,pos,clicked,extract,preprocess,WRich05_S,phrasal,PartsOfSpeech
0,1,d119db54-90aa-48b3-9035-eeda55edc38d,Incoming Webhooks — Mattermost 5.11 documentat...,0,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 6}","{'WRich05_S': 2.158255100250244, 'WRich10_S': ...","{'to_NoPhr_C': 10, 'as_NoPhr_C': 1.66666666666...","{'to_NoTag_C': 8.0, 'as_NoTag_C': 1.3333333333..."
1,2,d119db54-90aa-48b3-9035-eeda55edc38d,“Hello” Is Facebook's New Android-Only Social ...,1,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 4}","{'WRich05_S': 2.363802380859852, 'WRich10_S': ...","{'to_NoPhr_C': 14, 'as_NoPhr_C': 3.5, 'at_NoPh...","{'to_NoTag_C': 6.0, 'as_NoTag_C': 1.5, 'at_NoT..."
2,3,d119db54-90aa-48b3-9035-eeda55edc38d,Walkthrough | HTTP Event CollectorHEC tokens a...,4,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 41, 'n_sent': 5}","{'WRich05_S': 1.7482594717293978, 'WRich10_S':...","{'to_NoPhr_C': 15, 'as_NoPhr_C': 3.0, 'at_NoPh...","{'to_NoTag_C': 14.0, 'as_NoTag_C': 2.8, 'at_No..."
3,4,d119db54-90aa-48b3-9035-eeda55edc38d,College Scholarships - Scholarships.comMost co...,3,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 3}","{'WRich05_S': 1.779820341616869, 'WRich10_S': ...","{'to_NoPhr_C': 8, 'as_NoPhr_C': 2.666666666666...","{'to_NoTag_C': 11.0, 'as_NoTag_C': 3.666666666..."
4,5,d119db54-90aa-48b3-9035-eeda55edc38d,How to start sending email — Mailgun API docum...,2,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 5}","{'WRich05_S': 1.9575455449521542, 'WRich10_S':...","{'to_NoPhr_C': 9, 'as_NoPhr_C': 1.8, 'at_NoPhr...","{'to_NoTag_C': 5.0, 'as_NoTag_C': 1.0, 'at_NoT..."
...,...,...,...,...,...,...,...,...,...,...
10287,26377,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,"Science, Technology, Engineering, and Math, in...",1,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 27, 'n_sent': 5}","{'WRich05_S': 2.206125855445862, 'WRich10_S': ...","{'to_NoPhr_C': 17, 'as_NoPhr_C': 3.4, 'at_NoPh...","{'to_NoTag_C': 1.0, 'as_NoTag_C': 0.2, 'at_NoT..."
10288,26378,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Stem Kitchen SFSTEM Kitchen & Garden is a farm...,6,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 22, 'n_sent': 2}","{'WRich05_S': 2.277161478996277, 'WRich10_S': ...","{'to_NoPhr_C': 9, 'as_NoPhr_C': 4.5, 'at_NoPhr...","{'to_NoTag_C': 7.0, 'as_NoTag_C': 3.5, 'at_NoT..."
10289,26379,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Stem Definition & Meaning - Merriam-WebsterKid...,7,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 22, 'n_sent': 3}","{'WRich05_S': 2.4866977632045746, 'WRich10_S':...","{'to_NoPhr_C': 12, 'as_NoPhr_C': 4.0, 'at_NoPh...","{'to_NoTag_C': 6.0, 'as_NoTag_C': 2.0, 'at_NoT..."
10290,26380,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Driven By StemStem Holdings has positioned its...,8,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 19, 'n_sent': 2}","{'WRich05_S': 1.3848039507865906, 'WRich10_S':...","{'to_NoPhr_C': 10, 'as_NoPhr_C': 5.0, 'at_NoPh...","{'to_NoTag_C': 7.0, 'as_NoTag_C': 3.5, 'at_NoT..."


In [ ]:
data2['TradFeats'] = ''
for i in range(len(data2)):
    data2['TradFeats'][i] = data2['extract'][i].TraF_()
data2

,rid,qid,full,pos,clicked,extract,preprocess,WRich05_S,phrasal,PartsOfSpeech,TradFeats
0,1,d119db54-90aa-48b3-9035-eeda55edc38d,Incoming Webhooks — Mattermost 5.11 documentat...,0,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 6}","{'WRich05_S': 2.158255100250244, 'WRich10_S': ...","{'to_NoPhr_C': 10, 'as_NoPhr_C': 1.66666666666...","{'to_NoTag_C': 8.0, 'as_NoTag_C': 1.3333333333...","{'FleschG_S': 12.308870967741935, 'AutoRea_S':..."
1,2,d119db54-90aa-48b3-9035-eeda55edc38d,“Hello” Is Facebook's New Android-Only Social ...,1,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 4}","{'WRich05_S': 2.363802380859852, 'WRich10_S': ...","{'to_NoPhr_C': 14, 'as_NoPhr_C': 3.5, 'at_NoPh...","{'to_NoTag_C': 6.0, 'as_NoTag_C': 1.5, 'at_NoT...","{'FleschG_S': 9.89714285714286, 'AutoRea_S': 8..."
2,3,d119db54-90aa-48b3-9035-eeda55edc38d,Walkthrough | HTTP Event CollectorHEC tokens a...,4,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 41, 'n_sent': 5}","{'WRich05_S': 1.7482594717293978, 'WRich10_S':...","{'to_NoPhr_C': 15, 'as_NoPhr_C': 3.0, 'at_NoPh...","{'to_NoTag_C': 14.0, 'as_NoTag_C': 2.8, 'at_No...","{'FleschG_S': 7.754341463414637, 'AutoRea_S': ..."
3,4,d119db54-90aa-48b3-9035-eeda55edc38d,College Scholarships - Scholarships.comMost co...,3,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 3}","{'WRich05_S': 1.779820341616869, 'WRich10_S': ...","{'to_NoPhr_C': 8, 'as_NoPhr_C': 2.666666666666...","{'to_NoTag_C': 11.0, 'as_NoTag_C': 3.666666666...","{'FleschG_S': 15.864285714285717, 'AutoRea_S':..."
4,5,d119db54-90aa-48b3-9035-eeda55edc38d,How to start sending email — Mailgun API docum...,2,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 5}","{'WRich05_S': 1.9575455449521542, 'WRich10_S':...","{'to_NoPhr_C': 9, 'as_NoPhr_C': 1.8, 'at_NoPhr...","{'to_NoTag_C': 5.0, 'as_NoTag_C': 1.0, 'at_NoT...","{'FleschG_S': 7.002193548387098, 'AutoRea_S': ..."
...,...,...,...,...,...,...,...,...,...,...,...
10287,26377,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,"Science, Technology, Engineering, and Math, in...",1,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 27, 'n_sent': 5}","{'WRich05_S': 2.206125855445862, 'WRich10_S': ...","{'to_NoPhr_C': 17, 'as_NoPhr_C': 3.4, 'at_NoPh...","{'to_NoTag_C': 1.0, 'as_NoTag_C': 0.2, 'at_NoT...","{'FleschG_S': 11.864148148148153, 'AutoRea_S':..."
10288,26378,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Stem Kitchen SFSTEM Kitchen & Garden is a farm...,6,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 22, 'n_sent': 2}","{'WRich05_S': 2.277161478996277, 'WRich10_S': ...","{'to_NoPhr_C': 9, 'as_NoPhr_C': 4.5, 'at_NoPhr...","{'to_NoTag_C': 7.0, 'as_NoTag_C': 3.5, 'at_NoT...","{'FleschG_S': 11.22727272727273, 'AutoRea_S': ..."
10289,26379,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Stem Definition & Meaning - Merriam-WebsterKid...,7,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 22, 'n_sent': 3}","{'WRich05_S': 2.4866977632045746, 'WRich10_S':...","{'to_NoPhr_C': 12, 'as_NoPhr_C': 4.0, 'at_NoPh...","{'to_NoTag_C': 6.0, 'as_NoTag_C': 2.0, 'at_NoT...","{'FleschG_S': 11.942727272727272, 'AutoRea_S':..."
10290,26380,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Driven By StemStem Holdings has positioned its...,8,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 19, 'n_sent': 2}","{'WRich05_S': 1.3848039507865906, 'WRich10_S':...","{'to_NoPhr_C': 10, 'as_NoPhr_C': 5.0, 'at_NoPh...","{'to_NoTag_C': 7.0, 'as_NoTag_C': 3.5, 'at_NoT...","{'FleschG_S': 16.683421052631584, 'AutoRea_S':..."


In [ ]:
data2['TTRatio'] = ''
for i in range(len(data2)):
    data2['TTRatio'][i] = data2['extract'][i].TTRF_()
data2

,rid,qid,full,pos,clicked,extract,preprocess,WRich05_S,phrasal,PartsOfSpeech,TradFeats,TTRatio
0,1,d119db54-90aa-48b3-9035-eeda55edc38d,Incoming Webhooks — Mattermost 5.11 documentat...,0,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 6}","{'WRich05_S': 2.158255100250244, 'WRich10_S': ...","{'to_NoPhr_C': 10, 'as_NoPhr_C': 1.66666666666...","{'to_NoTag_C': 8.0, 'as_NoTag_C': 1.3333333333...","{'FleschG_S': 12.308870967741935, 'AutoRea_S':...","{'SimpTTR_S': 0.7096774193548387, 'CorrTTR_S':..."
1,2,d119db54-90aa-48b3-9035-eeda55edc38d,“Hello” Is Facebook's New Android-Only Social ...,1,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 4}","{'WRich05_S': 2.363802380859852, 'WRich10_S': ...","{'to_NoPhr_C': 14, 'as_NoPhr_C': 3.5, 'at_NoPh...","{'to_NoTag_C': 6.0, 'as_NoTag_C': 1.5, 'at_NoT...","{'FleschG_S': 9.89714285714286, 'AutoRea_S': 8...","{'SimpTTR_S': 0.7857142857142857, 'CorrTTR_S':..."
2,3,d119db54-90aa-48b3-9035-eeda55edc38d,Walkthrough | HTTP Event CollectorHEC tokens a...,4,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 41, 'n_sent': 5}","{'WRich05_S': 1.7482594717293978, 'WRich10_S':...","{'to_NoPhr_C': 15, 'as_NoPhr_C': 3.0, 'at_NoPh...","{'to_NoTag_C': 14.0, 'as_NoTag_C': 2.8, 'at_No...","{'FleschG_S': 7.754341463414637, 'AutoRea_S': ...","{'SimpTTR_S': 0.7804878048780488, 'CorrTTR_S':..."
3,4,d119db54-90aa-48b3-9035-eeda55edc38d,College Scholarships - Scholarships.comMost co...,3,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 3}","{'WRich05_S': 1.779820341616869, 'WRich10_S': ...","{'to_NoPhr_C': 8, 'as_NoPhr_C': 2.666666666666...","{'to_NoTag_C': 11.0, 'as_NoTag_C': 3.666666666...","{'FleschG_S': 15.864285714285717, 'AutoRea_S':...","{'SimpTTR_S': 0.7142857142857143, 'CorrTTR_S':..."
4,5,d119db54-90aa-48b3-9035-eeda55edc38d,How to start sending email — Mailgun API docum...,2,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 5}","{'WRich05_S': 1.9575455449521542, 'WRich10_S':...","{'to_NoPhr_C': 9, 'as_NoPhr_C': 1.8, 'at_NoPhr...","{'to_NoTag_C': 5.0, 'as_NoTag_C': 1.0, 'at_NoT...","{'FleschG_S': 7.002193548387098, 'AutoRea_S': ...","{'SimpTTR_S': 0.5806451612903226, 'CorrTTR_S':..."
...,...,...,...,...,...,...,...,...,...,...,...,...
10287,26377,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,"Science, Technology, Engineering, and Math, in...",1,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 27, 'n_sent': 5}","{'WRich05_S': 2.206125855445862, 'WRich10_S': ...","{'to_NoPhr_C': 17, 'as_NoPhr_C': 3.4, 'at_NoPh...","{'to_NoTag_C': 1.0, 'as_NoTag_C': 0.2, 'at_NoT...","{'FleschG_S': 11.864148148148153, 'AutoRea_S':...","{'SimpTTR_S': 0.6296296296296297, 'CorrTTR_S':..."
10288,26378,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Stem Kitchen SFSTEM Kitchen & Garden is a farm...,6,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 22, 'n_sent': 2}","{'WRich05_S': 2.277161478996277, 'WRich10_S': ...","{'to_NoPhr_C': 9, 'as_NoPhr_C': 4.5, 'at_NoPhr...","{'to_NoTag_C': 7.0, 'as_NoTag_C': 3.5, 'at_NoT...","{'FleschG_S': 11.22727272727273, 'AutoRea_S': ...","{'SimpTTR_S': 0.8181818181818182, 'CorrTTR_S':..."
10289,26379,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Stem Definition & Meaning - Merriam-WebsterKid...,7,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 22, 'n_sent': 3}","{'WRich05_S': 2.4866977632045746, 'WRich10_S':...","{'to_NoPhr_C': 12, 'as_NoPhr_C': 4.0, 'at_NoPh...","{'to_NoTag_C': 6.0, 'as_NoTag_C': 2.0, 'at_NoT...","{'FleschG_S': 11.942727272727272, 'AutoRea_S':...","{'SimpTTR_S': 0.5909090909090909, 'CorrTTR_S':..."
10290,26380,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Driven By StemStem Holdings has positioned its...,8,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 19, 'n_sent': 2}","{'WRich05_S': 1.3848039507865906, 'WRich10_S':...","{'to_NoPhr_C': 10, 'as_NoPhr_C': 5.0, 'at_NoPh...","{'to_NoTag_C': 7.0, 'as_NoTag_C': 3.5, 'at_NoT..."

In [ ]:
data2['Variation'] = ''
for i in range(len(data2)):
    data2['Variation'][i] = data2['extract'][i].VarF_()
data2

,rid,qid,full,pos,clicked,extract,preprocess,WRich05_S,phrasal,PartsOfSpeech,TradFeats,TTRatio,Variation
0,1,d119db54-90aa-48b3-9035-eeda55edc38d,Incoming Webhooks — Mattermost 5.11 documentat...,0,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 6}","{'WRich05_S': 2.158255100250244, 'WRich10_S': ...","{'to_NoPhr_C': 10, 'as_NoPhr_C': 1.66666666666...","{'to_NoTag_C': 8.0, 'as_NoTag_C': 1.3333333333...","{'FleschG_S': 12.308870967741935, 'AutoRea_S':...","{'SimpTTR_S': 0.7096774193548387, 'CorrTTR_S':...","{'SimpNoV_S': 0.625, 'SquaNoV_S': 3.125, 'Corr..."
1,2,d119db54-90aa-48b3-9035-eeda55edc38d,“Hello” Is Facebook's New Android-Only Social ...,1,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 4}","{'WRich05_S': 2.363802380859852, 'WRich10_S': ...","{'to_NoPhr_C': 14, 'as_NoPhr_C': 3.5, 'at_NoPh...","{'to_NoTag_C': 6.0, 'as_NoTag_C': 1.5, 'at_NoT...","{'FleschG_S': 9.89714285714286, 'AutoRea_S': 8...","{'SimpTTR_S': 0.7857142857142857, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 6.0, 'CorrNoV_..."
2,3,d119db54-90aa-48b3-9035-eeda55edc38d,Walkthrough | HTTP Event CollectorHEC tokens a...,4,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 41, 'n_sent': 5}","{'WRich05_S': 1.7482594717293978, 'WRich10_S':...","{'to_NoPhr_C': 15, 'as_NoPhr_C': 3.0, 'at_NoPh...","{'to_NoTag_C': 14.0, 'as_NoTag_C': 2.8, 'at_No...","{'FleschG_S': 7.754341463414637, 'AutoRea_S': ...","{'SimpTTR_S': 0.7804878048780488, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 14.0, 'CorrNoV..."
3,4,d119db54-90aa-48b3-9035-eeda55edc38d,College Scholarships - Scholarships.comMost co...,3,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 3}","{'WRich05_S': 1.779820341616869, 'WRich10_S': ...","{'to_NoPhr_C': 8, 'as_NoPhr_C': 2.666666666666...","{'to_NoTag_C': 11.0, 'as_NoTag_C': 3.666666666...","{'FleschG_S': 15.864285714285717, 'AutoRea_S':...","{'SimpTTR_S': 0.7142857142857143, 'CorrTTR_S':...","{'SimpNoV_S': 0.5454545454545454, 'SquaNoV_S':..."
4,5,d119db54-90aa-48b3-9035-eeda55edc38d,How to start sending email — Mailgun API docum...,2,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 5}","{'WRich05_S': 1.9575455449521542, 'WRich10_S':...","{'to_NoPhr_C': 9, 'as_NoPhr_C': 1.8, 'at_NoPhr...","{'to_NoTag_C': 5.0, 'as_NoTag_C': 1.0, 'at_NoT...","{'FleschG_S': 7.002193548387098, 'AutoRea_S': ...","{'SimpTTR_S': 0.5806451612903226, 'CorrTTR_S':...","{'SimpNoV_S': 0.4, 'SquaNoV_S': 0.8, 'CorrNoV_..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,26377,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,"Science, Technology, Engineering, and Math, in...",1,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 27, 'n_sent': 5}","{'WRich05_S': 2.206125855445862, 'WRich10_S': ...","{'to_NoPhr_C': 17, 'as_NoPhr_C': 3.4, 'at_NoPh...","{'to_NoTag_C': 1.0, 'as_NoTag_C': 0.2, 'at_NoT...","{'FleschG_S': 11.864148148148153, 'AutoRea_S':...","{'SimpTTR_S': 0.6296296296296297, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 1.0, 'CorrNoV_..."
10288,26378,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Stem Kitchen SFSTEM Kitchen & Garden is a farm...,6,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 22, 'n_sent': 2}","{'WRich05_S': 2.277161478996277, 'WRich10_S': ...","{'to_NoPhr_C': 9, 'as_NoPhr_C': 4.5, 'at_NoPhr...","{'to_NoTag_C': 7.0, 'as_NoTag_C': 3.5, 'at_NoT...","{'FleschG_S': 11.22727272727273, 'AutoRea_S': ...","{'SimpTTR_S': 0.8181818181818182, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 7.0, 'CorrNoV_..."
10289,26379,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Stem Definition & Meaning - Merriam-WebsterKid...,7,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 22, 'n_sent': 3}","{'WRich05_S': 2.4866977632045746, 'WRich10_S':...","{'to_NoPhr_C': 12, 'as_NoPhr_C': 4.0, 'at_NoPh...","{'to_NoTag_C': 6.0, 'as_NoTag_C': 2.0, 'at_NoT...","{'FleschG_S': 11.942727272727272, 'AutoRea_S':...","{'SimpTTR_S'

In [ ]:
data2['PDifficulty'] = ''
for i in range(len(data2)):
    data2['PDifficulty'][i] = data2['extract'][i].PsyF_()
data2

,rid,qid,full,pos,clicked,extract,preprocess,WRich05_S,phrasal,PartsOfSpeech,TradFeats,TTRatio,Variation,PDifficulty
0,1,d119db54-90aa-48b3-9035-eeda55edc38d,Incoming Webhooks — Mattermost 5.11 documentat...,0,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 6}","{'WRich05_S': 2.158255100250244, 'WRich10_S': ...","{'to_NoPhr_C': 10, 'as_NoPhr_C': 1.66666666666...","{'to_NoTag_C': 8.0, 'as_NoTag_C': 1.3333333333...","{'FleschG_S': 12.308870967741935, 'AutoRea_S':...","{'SimpTTR_S': 0.7096774193548387, 'CorrTTR_S':...","{'SimpNoV_S': 0.625, 'SquaNoV_S': 3.125, 'Corr...","{'to_AAKuW_C': 156.19, 'as_AAKuW_C': 26.031666..."
1,2,d119db54-90aa-48b3-9035-eeda55edc38d,“Hello” Is Facebook's New Android-Only Social ...,1,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 4}","{'WRich05_S': 2.363802380859852, 'WRich10_S': ...","{'to_NoPhr_C': 14, 'as_NoPhr_C': 3.5, 'at_NoPh...","{'to_NoTag_C': 6.0, 'as_NoTag_C': 1.5, 'at_NoT...","{'FleschG_S': 9.89714285714286, 'AutoRea_S': 8...","{'SimpTTR_S': 0.7857142857142857, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 6.0, 'CorrNoV_...","{'to_AAKuW_C': 145.72000000000003, 'as_AAKuW_C..."
2,3,d119db54-90aa-48b3-9035-eeda55edc38d,Walkthrough | HTTP Event CollectorHEC tokens a...,4,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 41, 'n_sent': 5}","{'WRich05_S': 1.7482594717293978, 'WRich10_S':...","{'to_NoPhr_C': 15, 'as_NoPhr_C': 3.0, 'at_NoPh...","{'to_NoTag_C': 14.0, 'as_NoTag_C': 2.8, 'at_No...","{'FleschG_S': 7.754341463414637, 'AutoRea_S': ...","{'SimpTTR_S': 0.7804878048780488, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 14.0, 'CorrNoV...","{'to_AAKuW_C': 218.05, 'as_AAKuW_C': 43.61, 'a..."
3,4,d119db54-90aa-48b3-9035-eeda55edc38d,College Scholarships - Scholarships.comMost co...,3,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 3}","{'WRich05_S': 1.779820341616869, 'WRich10_S': ...","{'to_NoPhr_C': 8, 'as_NoPhr_C': 2.666666666666...","{'to_NoTag_C': 11.0, 'as_NoTag_C': 3.666666666...","{'FleschG_S': 15.864285714285717, 'AutoRea_S':...","{'SimpTTR_S': 0.7142857142857143, 'CorrTTR_S':...","{'SimpNoV_S': 0.5454545454545454, 'SquaNoV_S':...","{'to_AAKuW_C': 189.13000000000002, 'as_AAKuW_C..."
4,5,d119db54-90aa-48b3-9035-eeda55edc38d,How to start sending email — Mailgun API docum...,2,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 5}","{'WRich05_S': 1.9575455449521542, 'WRich10_S':...","{'to_NoPhr_C': 9, 'as_NoPhr_C': 1.8, 'at_NoPhr...","{'to_NoTag_C': 5.0, 'as_NoTag_C': 1.0, 'at_NoT...","{'FleschG_S': 7.002193548387098, 'AutoRea_S': ...","{'SimpTTR_S': 0.5806451612903226, 'CorrTTR_S':...","{'SimpNoV_S': 0.4, 'SquaNoV_S': 0.8, 'CorrNoV_...","{'to_AAKuW_C': 167.00000000000003, 'as_AAKuW_C..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,26377,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,"Science, Technology, Engineering, and Math, in...",1,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 27, 'n_sent': 5}","{'WRich05_S': 2.206125855445862, 'WRich10_S': ...","{'to_NoPhr_C': 17, 'as_NoPhr_C': 3.4, 'at_NoPh...","{'to_NoTag_C': 1.0, 'as_NoTag_C': 0.2, 'at_NoT...","{'FleschG_S': 11.864148148148153, 'AutoRea_S':...","{'SimpTTR_S': 0.6296296296296297, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 1.0, 'CorrNoV_...","{'to_AAKuW_C': 176.52, 'as_AAKuW_C': 35.304, '..."
10288,26378,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Stem Kitchen SFSTEM Kitchen & Garden is a farm...,6,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 22, 'n_sent': 2}","{'WRich05_S': 2.277161478996277, 'WRich10_S': ...","{'to_NoPhr_C': 9, 'as_NoPhr_C': 4.5, 'at_NoPhr...","{'to_NoTag_C': 7.0, 'as_NoTag_C': 3.5, 'at_NoT...","{'FleschG_S': 11.22727272727273, 'AutoRea_S': ...","{'SimpTTR_S': 0.8181818181818182, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 7.0, 'CorrNoV_...","{'to_AAKuW_C': 117.22999999999999, 'as_AAKuW_C..."
10289,26379,73308e25-c3e2

In [ ]:
data2['TRSF'] = ''
for i in range(len(data2)):
    data2['TRSF'][i] = data2['extract'][i].TrSF_()
data2

,rid,qid,full,pos,clicked,extract,preprocess,WRich05_S,phrasal,PartsOfSpeech,TradFeats,TTRatio,Variation,PDifficulty,TRSF
0,1,d119db54-90aa-48b3-9035-eeda55edc38d,Incoming Webhooks — Mattermost 5.11 documentat...,0,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 6}","{'WRich05_S': 2.158255100250244, 'WRich10_S': ...","{'to_NoPhr_C': 10, 'as_NoPhr_C': 1.66666666666...","{'to_NoTag_C': 8.0, 'as_NoTag_C': 1.3333333333...","{'FleschG_S': 12.308870967741935, 'AutoRea_S':...","{'SimpTTR_S': 0.7096774193548387, 'CorrTTR_S':...","{'SimpNoV_S': 0.625, 'SquaNoV_S': 3.125, 'Corr...","{'to_AAKuW_C': 156.19, 'as_AAKuW_C': 26.031666...","{'to_TreeH_C': 35, 'as_TreeH_C': 5.83333333333..."
1,2,d119db54-90aa-48b3-9035-eeda55edc38d,“Hello” Is Facebook's New Android-Only Social ...,1,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 4}","{'WRich05_S': 2.363802380859852, 'WRich10_S': ...","{'to_NoPhr_C': 14, 'as_NoPhr_C': 3.5, 'at_NoPh...","{'to_NoTag_C': 6.0, 'as_NoTag_C': 1.5, 'at_NoT...","{'FleschG_S': 9.89714285714286, 'AutoRea_S': 8...","{'SimpTTR_S': 0.7857142857142857, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 6.0, 'CorrNoV_...","{'to_AAKuW_C': 145.72000000000003, 'as_AAKuW_C...","{'to_TreeH_C': 18, 'as_TreeH_C': 4.5, 'at_Tree..."
2,3,d119db54-90aa-48b3-9035-eeda55edc38d,Walkthrough | HTTP Event CollectorHEC tokens a...,4,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 41, 'n_sent': 5}","{'WRich05_S': 1.7482594717293978, 'WRich10_S':...","{'to_NoPhr_C': 15, 'as_NoPhr_C': 3.0, 'at_NoPh...","{'to_NoTag_C': 14.0, 'as_NoTag_C': 2.8, 'at_No...","{'FleschG_S': 7.754341463414637, 'AutoRea_S': ...","{'SimpTTR_S': 0.7804878048780488, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 14.0, 'CorrNoV...","{'to_AAKuW_C': 218.05, 'as_AAKuW_C': 43.61, 'a...","{'to_TreeH_C': 31, 'as_TreeH_C': 6.2, 'at_Tree..."
3,4,d119db54-90aa-48b3-9035-eeda55edc38d,College Scholarships - Scholarships.comMost co...,3,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 3}","{'WRich05_S': 1.779820341616869, 'WRich10_S': ...","{'to_NoPhr_C': 8, 'as_NoPhr_C': 2.666666666666...","{'to_NoTag_C': 11.0, 'as_NoTag_C': 3.666666666...","{'FleschG_S': 15.864285714285717, 'AutoRea_S':...","{'SimpTTR_S': 0.7142857142857143, 'CorrTTR_S':...","{'SimpNoV_S': 0.5454545454545454, 'SquaNoV_S':...","{'to_AAKuW_C': 189.13000000000002, 'as_AAKuW_C...","{'to_TreeH_C': 21, 'as_TreeH_C': 7.0, 'at_Tree..."
4,5,d119db54-90aa-48b3-9035-eeda55edc38d,How to start sending email — Mailgun API docum...,2,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 5}","{'WRich05_S': 1.9575455449521542, 'WRich10_S':...","{'to_NoPhr_C': 9, 'as_NoPhr_C': 1.8, 'at_NoPhr...","{'to_NoTag_C': 5.0, 'as_NoTag_C': 1.0, 'at_NoT...","{'FleschG_S': 7.002193548387098, 'AutoRea_S': ...","{'SimpTTR_S': 0.5806451612903226, 'CorrTTR_S':...","{'SimpNoV_S': 0.4, 'SquaNoV_S': 0.8, 'CorrNoV_...","{'to_AAKuW_C': 167.00000000000003, 'as_AAKuW_C...","{'to_TreeH_C': 28, 'as_TreeH_C': 5.6, 'at_Tree..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,26377,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,"Science, Technology, Engineering, and Math, in...",1,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 27, 'n_sent': 5}","{'WRich05_S': 2.206125855445862, 'WRich10_S': ...","{'to_NoPhr_C': 17, 'as_NoPhr_C': 3.4, 'at_NoPh...","{'to_NoTag_C': 1.0, 'as_NoTag_C': 0.2, 'at_NoT...","{'FleschG_S': 11.864148148148153, 'AutoRea_S':...","{'SimpTTR_S': 0.6296296296296297, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 1.0, 'CorrNoV_...","{'to_AAKuW_C': 176.52, 'as_AAKuW_C': 35.304, '...","{'to_TreeH_C': 24, 'as_TreeH_C': 4.8, 'at_Tree..."
10288,26378,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,Stem Kitchen SFSTEM Kitchen & Garden is a farm...,6,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 22, 'n_sent': 2}","{'WRich05_S': 2.277161478996277, 'WRich10_S': ...","{'to_NoPhr_C': 

In [ ]:
data2['shallow'] = ''
for i in range(len(data2)):
    data2['shallow'][i] = data2['extract'][i].ShaF_()
data2

,rid,qid,full,pos,clicked,extract,preprocess,WRich05_S,phrasal,PartsOfSpeech,TradFeats,TTRatio,Variation,PDifficulty,TRSF,shallow
0,1,d119db54-90aa-48b3-9035-eeda55edc38d,Incoming Webhooks — Mattermost 5.11 documentat...,0,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 6}","{'WRich05_S': 2.158255100250244, 'WRich10_S': ...","{'to_NoPhr_C': 10, 'as_NoPhr_C': 1.66666666666...","{'to_NoTag_C': 8.0, 'as_NoTag_C': 1.3333333333...","{'FleschG_S': 12.308870967741935, 'AutoRea_S':...","{'SimpTTR_S': 0.7096774193548387, 'CorrTTR_S':...","{'SimpNoV_S': 0.625, 'SquaNoV_S': 3.125, 'Corr...","{'to_AAKuW_C': 156.19, 'as_AAKuW_C': 26.031666...","{'to_TreeH_C': 35, 'as_TreeH_C': 5.83333333333...","{'TokSenM_S': 186.0, 'TokSenS_S': 13.638181696..."
1,2,d119db54-90aa-48b3-9035-eeda55edc38d,“Hello” Is Facebook's New Android-Only Social ...,1,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 4}","{'WRich05_S': 2.363802380859852, 'WRich10_S': ...","{'to_NoPhr_C': 14, 'as_NoPhr_C': 3.5, 'at_NoPh...","{'to_NoTag_C': 6.0, 'as_NoTag_C': 1.5, 'at_NoT...","{'FleschG_S': 9.89714285714286, 'AutoRea_S': 8...","{'SimpTTR_S': 0.7857142857142857, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 6.0, 'CorrNoV_...","{'to_AAKuW_C': 145.72000000000003, 'as_AAKuW_C...","{'to_TreeH_C': 18, 'as_TreeH_C': 4.5, 'at_Tree...","{'TokSenM_S': 112.0, 'TokSenS_S': 10.583005244..."
2,3,d119db54-90aa-48b3-9035-eeda55edc38d,Walkthrough | HTTP Event CollectorHEC tokens a...,4,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 41, 'n_sent': 5}","{'WRich05_S': 1.7482594717293978, 'WRich10_S':...","{'to_NoPhr_C': 15, 'as_NoPhr_C': 3.0, 'at_NoPh...","{'to_NoTag_C': 14.0, 'as_NoTag_C': 2.8, 'at_No...","{'FleschG_S': 7.754341463414637, 'AutoRea_S': ...","{'SimpTTR_S': 0.7804878048780488, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 14.0, 'CorrNoV...","{'to_AAKuW_C': 218.05, 'as_AAKuW_C': 43.61, 'a...","{'to_TreeH_C': 31, 'as_TreeH_C': 6.2, 'at_Tree...","{'TokSenM_S': 205.0, 'TokSenS_S': 14.317821063..."
3,4,d119db54-90aa-48b3-9035-eeda55edc38d,College Scholarships - Scholarships.comMost co...,3,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 28, 'n_sent': 3}","{'WRich05_S': 1.779820341616869, 'WRich10_S': ...","{'to_NoPhr_C': 8, 'as_NoPhr_C': 2.666666666666...","{'to_NoTag_C': 11.0, 'as_NoTag_C': 3.666666666...","{'FleschG_S': 15.864285714285717, 'AutoRea_S':...","{'SimpTTR_S': 0.7142857142857143, 'CorrTTR_S':...","{'SimpNoV_S': 0.5454545454545454, 'SquaNoV_S':...","{'to_AAKuW_C': 189.13000000000002, 'as_AAKuW_C...","{'to_TreeH_C': 21, 'as_TreeH_C': 7.0, 'at_Tree...","{'TokSenM_S': 84.0, 'TokSenS_S': 9.16515138991..."
4,5,d119db54-90aa-48b3-9035-eeda55edc38d,How to start sending email — Mailgun API docum...,2,0,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 31, 'n_sent': 5}","{'WRich05_S': 1.9575455449521542, 'WRich10_S':...","{'to_NoPhr_C': 9, 'as_NoPhr_C': 1.8, 'at_NoPhr...","{'to_NoTag_C': 5.0, 'as_NoTag_C': 1.0, 'at_NoT...","{'FleschG_S': 7.002193548387098, 'AutoRea_S': ...","{'SimpTTR_S': 0.5806451612903226, 'CorrTTR_S':...","{'SimpNoV_S': 0.4, 'SquaNoV_S': 0.8, 'CorrNoV_...","{'to_AAKuW_C': 167.00000000000003, 'as_AAKuW_C...","{'to_TreeH_C': 28, 'as_TreeH_C': 5.6, 'at_Tree...","{'TokSenM_S': 155.0, 'TokSenS_S': 12.449899597..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,26377,73308e25-c3e2-4fd6-ac7c-91a88c2b6c41,"Science, Technology, Engineering, and Math, in...",1,1,<lingfeat.extractor.pass_text object at 0x0000...,"{'n_token': 27, 'n_sent': 5}","{'WRich05_S': 2.206125855445862, 'WRich10_S': ...","{'to_NoPhr_C': 17, 'as_NoPhr_C': 3.4, 'at_NoPh...","{'to_NoTag_C': 1.0, 'as_NoTag_C': 0.2, 'at_NoT...","{'FleschG_S': 11.864148148148153, 'AutoRea_S':...","{'SimpTTR_S': 0.6296296296296297, 'CorrTTR_S':...","{'SimpNoV_S': 1.0, 'SquaNoV_S': 1.0, 'CorrNoV_...","{'to_AAKuW_C': 176.52, 'as_AAKuW_C': 35.304, '...","{'to_TreeH_C': 24, 'as_TreeH_C'

In [ ]:
data2.to_csv('features.csv')

#### Example of accessing the values

In [ ]:
ttratio=data2['TTRatio'][0]
ttratio
#ttratioKey = list(ttratio)
#for key in ttratioKey:
    #print(ttratio[key])

{'SimpTTR_S': 0.7096774193548387,
 'CorrTTR_S': 2.794002794004191,
 'BiLoTTR_S': 0.9001321988972736,
 'UberTTR_S': 27.860299412863338,
 'MTLDTTR_S': 0.0}

In [ ]:
test = data2['phrasal'][0] #these are dictionaries
testKeys = list(test) #get the keys
for key in testKeys:
    print(test[key])

10
1.6666666666666667
0.3225806451612903
1.0
10.0
5.0
5.0
0.0
10
1.6666666666666667
0.3225806451612903
1.0
10.0
5.0
5.0
0.0
1
0.16666666666666666
0.03225806451612903
0.1
0.1
0.5
0.5
0.0
2
0.3333333333333333
0.06451612903225806
0.2
0.2
2.0
1.0
0.0
2
0.3333333333333333
0.06451612903225806
0.2
0.2
2.0
1.0
0.0
0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
